In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import subprocess
import os.path
import time
import math
import csv
#%matplotlib inline
    
from sklearn.datasets import make_regression, make_classification, load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical,np_utils
from keras.callbacks import EarlyStopping

#Function from Alex Lin 
def bash(string):
    '''run a bash command and return stdout lines as a py list'''
    stdout=(subprocess.Popen(string, shell=True, stdout=subprocess.PIPE).stdout.read())
    output = str(stdout)[2:-1].split('\\n')[:-1]
    #output = str(stdout).split('\\n')[1:-1]
    #output[0] = output[0][2:]
    return output

# Get all the spectrograms in the Data Folder
# then put in a numpy array
# Get labels too

def load_data(data_folder, divisor, start_at = "none", stop = "none"):
    lis = bash('ls '+ data_folder)
    
    start_flag = 0
    img_array = []
    labels = []
    
    if start_at == "none":
        start_flag = 1

    for path in lis:
        path_ = data_folder+ path
        
        if path_ == start_at:
            start_flag = 1
        
        if start_flag == 0:
            continue

        if path_ == stop:
            break;

        img = cv2.imread(path_,1)
        img_norm = img/255
        img_norm.resize(int(513/divisor),int(800/divisor),3)
        img_array.append(img_norm)
        labels.append(path[0:6])
    return img_array, labels

def convert_img_array_to_X(img_array):
    X = np.array(img_array)
    return X

def labels_onehot(labels):
    le = LabelEncoder()
    y_cat = le.fit_transform(labels)
    y_cat = y_cat.reshape(-1,1)
    
    ohe = OneHotEncoder()
    y_cat = ohe.fit_transform(y_cat).toarray()
    
    return y_cat

Using TensorFlow backend.


In [3]:
model = load_model('model_03_9ppl_v1.h5')

In [4]:
model3 = Model(inputs=model.input, outputs=model.get_layer('flatten_1').output)

Load Data for Train , Test

In [5]:
X3 ,y3 = load_data(data_folder="../Data/", divisor = 2,start_at = '../Data/20_mp3-000.png',stop = '../Data/24_mp3-000.png')

X3 = convert_img_array_to_X(X3)
y3 = labels_onehot(y3)

Train-Test Split

In [6]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3,y3,test_size = 0.1,stratify = y3)

Initialize SVM

In [7]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', class_weight='balanced')

X_train3_predict = model3.predict(X_train3)
X_test3_predict = model3.predict(X_test3)

y_train3_predict = [np.where(r==1)[0][0] for r in y_train3]
y_test3_predict = [np.where(r==1)[0][0] for r in y_test3]

In [8]:
len(y_train3)
y_train3.shape

clf.fit(X_train3_predict, y_train3_predict)

predictions = clf.predict(X_test3_predict)

from sklearn.metrics import accuracy_score
accuracy_score(y_test3_predict, predictions)

confusion_matrix(y_test3_predict, predictions)

array([[46,  0,  0],
       [ 0, 23,  0],
       [ 0, 12, 29]])

In [8]:
from sklearn import svm
clf = svm.SVC(kernel='rbf', class_weight='balanced')

X_train3_predict = model3.predict(X_train3)
X_test3_predict = model3.predict(X_test3)

y_train3_predict = [np.where(r==1)[0][0] for r in y_train3]
y_test3_predict = [np.where(r==1)[0][0] for r in y_test3]

y_train3.shape

clf.fit(X_train3_predict, y_train3_predict)

predictions = clf.predict(X_test3_predict)

from sklearn.metrics import accuracy_score
accuracy_score(y_test3_predict, predictions)

confusion_matrix(y_test3_predict, predictions)

array([[44,  2,  0],
       [ 0, 43,  0],
       [ 2,  6, 43]])